In [4]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [5]:
# 불러올 파일 - 장소 정보 데이터
name_data = pd.read_csv(f'gg_search_sample.csv', encoding='utf-8-sig')

# 저장할 파일경로
save_file = f'gg_review_sample.csv'

In [6]:
print(len(name_data))
name_data.head(3)

43


,title,link,category,description,telephone,address,roadAddress,mapx,mapy,search_input
0,토담골,NaN,"한식>백반,가정식",NaN,NaN,경기도 수원시 장안구 파장동 578-18 1층,경기도 수원시 장안구 경수대로1073번길 24 1층,311034,523384,경기도 수원시 장안구 파장동 가정식
1,현정이네식당,NaN,"한식>백반,가정식",NaN,NaN,경기도 수원시 장안구 파장동 595-16,경기도 수원시 장안구 경수대로1031번길 8 한국일보,311442,523255,경기도 수원시 장안구 파장동 가정식
2,이목식당,NaN,"한식>백반,가정식",NaN,NaN,경기도 수원시 장안구 이목동 210-10,경기도 수원시 장안구 장안로 433,309873,524501,경기도 수원시 장안구 파장동 가정식


In [7]:
def stores_name(name):
    url_start = "https://m.map.naver.com/search2/search.naver"
    url_query = "?query="
    url_end = "&sm=hty&style=v5#/list"
    url_store = name.replace(' ','%20')
    return url_start+url_query+url_store+url_end

In [8]:
name_data['stores_url'] = name_data['title'].apply(stores_name)
name_data.head(3)

,title,link,category,description,telephone,address,roadAddress,mapx,mapy,search_input,stores_url
0,토담골,NaN,"한식>백반,가정식",NaN,NaN,경기도 수원시 장안구 파장동 578-18 1층,경기도 수원시 장안구 경수대로1073번길 24 1층,311034,523384,경기도 수원시 장안구 파장동 가정식,https://m.map.naver.com/search2/search.naver?q...
1,현정이네식당,NaN,"한식>백반,가정식",NaN,NaN,경기도 수원시 장안구 파장동 595-16,경기도 수원시 장안구 경수대로1031번길 8 한국일보,311442,523255,경기도 수원시 장안구 파장동 가정식,https://m.map.naver.com/search2/search.naver?q...
2,이목식당,NaN,"한식>백반,가정식",NaN,NaN,경기도 수원시 장안구 이목동 210-10,경기도 수원시 장안구 장안로 433,309873,524501,경기도 수원시 장안구 파장동 가정식,https://m.map.naver.com/search2/search.naver?q...


In [9]:
# NaN -> ' ' 공백으로
name_data = name_data.fillna(' ')

In [10]:
############# 크롤링 할 갯수 입력 a ~ b-1 --> [a:b] #################
name_data = name_data[:5].reset_index(drop=True) # 샘플 5개
name_data

,title,link,category,description,telephone,address,roadAddress,mapx,mapy,search_input,stores_url
0,토담골,,"한식>백반,가정식",,,경기도 수원시 장안구 파장동 578-18 1층,경기도 수원시 장안구 경수대로1073번길 24 1층,311034,523384,경기도 수원시 장안구 파장동 가정식,https://m.map.naver.com/search2/search.naver?q...
1,현정이네식당,,"한식>백반,가정식",,,경기도 수원시 장안구 파장동 595-16,경기도 수원시 장안구 경수대로1031번길 8 한국일보,311442,523255,경기도 수원시 장안구 파장동 가정식,https://m.map.naver.com/search2/search.naver?q...
2,이목식당,,"한식>백반,가정식",,,경기도 수원시 장안구 이목동 210-10,경기도 수원시 장안구 장안로 433,309873,524501,경기도 수원시 장안구 파장동 가정식,https://m.map.naver.com/search2/search.naver?q...
3,엄마밥상,,"한식>백반,가정식",,,경기도 수원시 장안구 파장동 267-1,경기도 수원시 장안구 경수대로1078번길 17-1 경진만물전자,311107,523601,경기도 수원시 장안구 파장동 가정식,https://m.map.naver.com/search2/search.naver?q...
4,행복한밥상,,"한식>백반,가정식",,,경기도 수원시 장안구 조원동 774-2,경기도 수원시 장안구 송정로 164,312842,522231,경기도 수원시 장안구 파장동 가정식,https://m.map.naver.com/search2/search.naver?q...


In [121]:
#검색할 식당 데이터와 url을 담을 데이터 프레임 생성
df = pd.DataFrame(columns=['name', 'category', 'address', 'roadAddress','naverURL'])

#데이터 프레임이 잘 만들어졌는지 확인
df['name'] = name_data['title']
df['category'] = name_data['category']
df['address'] = name_data['address']
df['roadAddress'] = name_data['roadAddress']
df['naverURL'] = name_data['stores_url']
df

,name,category,address,roadAddress,naverURL
0,토담골,"한식>백반,가정식",경기도 수원시 장안구 파장동 578-18 1층,경기도 수원시 장안구 경수대로1073번길 24 1층,https://m.map.naver.com/search2/search.naver?q...
1,현정이네식당,"한식>백반,가정식",경기도 수원시 장안구 파장동 595-16,경기도 수원시 장안구 경수대로1031번길 8 한국일보,https://m.map.naver.com/search2/search.naver?q...
2,이목식당,"한식>백반,가정식",경기도 수원시 장안구 이목동 210-10,경기도 수원시 장안구 장안로 433,https://m.map.naver.com/search2/search.naver?q...
3,엄마밥상,"한식>백반,가정식",경기도 수원시 장안구 파장동 267-1,경기도 수원시 장안구 경수대로1078번길 17-1 경진만물전자,https://m.map.naver.com/search2/search.naver?q...
4,행복한밥상,"한식>백반,가정식",경기도 수원시 장안구 조원동 774-2,경기도 수원시 장안구 송정로 164,https://m.map.naver.com/search2/search.naver?q...


In [122]:
# df 전체에서 결측값이 있으면 해당 행 삭제
df = df.dropna()
df.loc[df['naverURL'].isna()]

,name,category,address,roadAddress,naverURL


In [123]:
# csv 파일로도 저장하여 url이 빈 값이 있는지 반드시 확인할 것 !
# url이 없으면 코드 실행이 중단되므로 반드시 url 데이터를 확인할 것!!
# df.to_csv('pretest_url.csv', encoding='utf-8-sig')

In [124]:
len(df)

5

In [125]:
# 해당 xpath 존재여부 체크 함수
def check_exists_by_xpath(xpath):
    try:
        driver.find_element(By.XPATH, (xpath))
    except NoSuchElementException:
        return False
    return True

In [126]:
def check_exists_by_img(xpath):
    try:
        driver.find_element(By.XPATH, (xpath))
    except NoSuchElementException:
        return False
    return True

In [127]:
# # 가게별 리뷰를 한줄로 합쳐줌
# def group_review(df):
#     df = df.fillna(' ')
#     df = df.groupby(['name', 'category','address', 'roadAddress','rate'])['review'].apply(lambda x: x.sum())
#     df = pd.DataFrame(df).reset_index()
#     return df

In [128]:
def open_window():
    # 창 오픈 
    # 창 최소화 시키면 안됨! 내리고 싶으면 백그라운드 실행!!
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    options.add_argument('headless')  # 크롬창 백그라운드 실행
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_window_size(500, 1000) # 윈도우 사이즈
    # driver.set_window_position(-525, 125) # 윈도우 오픈 포지션 - 서브모니터
    time.sleep(3)
    return driver

In [129]:
driver = open_window()

current = 0 #현재 진행 상황

goal = len(df['name']) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]


for i in range(len(df)): 
    current += 1
    print('***** 진행상황 : ', current,'/', goal, sep="")
    # n회마다 리프레시
    n = 100
    if (current != 1) and (current % n == 0):
        print("창을 재실행 합니다...")
        driver.close()
        driver = open_window()
        # time.sleep(10) # 안넣어도됨, 어차피 알아서 열릴때까지 기다림

    
    # 스토어 검색 url 접속
    driver.get(df['naverURL'][i]) 
    time.sleep(2)
    
    # 검색어 최상단 스토어 클릭
    # print('대표 사진 여부 -> xpath 위치 결정')
    if len(driver.find_elements(By.XPATH, '//*[@id="ct"]/div[2]/ul/li/div[1]')) >= 1:
        if check_exists_by_img('//*[@id="ct"]/div[2]/ul/li/div[1]/a[1]/img') == True:
            #check_exists_by_xpath('//*[@id="ct"]/div[2]/ul/li/div[1]/a[2]/div/strong') != False:
            button = driver.find_element(By.XPATH,('//*[@id="ct"]/div[2]/ul/li/div[1]/a[2]/div/strong'))
            button.click()
            time.sleep(2)
        else:
            button = driver.find_element(By.XPATH,('//*[@id="ct"]/div[2]/ul/li/div[1]/a/div/strong'))
            button.click()
            time.sleep(2)
    else: 
        print('가게 존재 X')
    time.sleep(2)

    # 현재 url (home_url)
    curr_url = driver.current_url
    home_url = curr_url
    
    # 현재 url의 앞쪽을 불러옴
    curr_url = curr_url[:-4]

    # 리뷰 url
    review_url = curr_url + 'review/visitor'

    # 리뷰 url 로 이동
    driver.get(review_url)

    # thisurl = df['naverURL'][i] #
    time.sleep(2)
    print('현재 수집중인 식당 : ', df['name'][i])
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(0.5)
    # driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
    
    #리뷰 더보기 버튼 누르기
    more_cnt = 0
    while more_cnt < 4: # 더보기 버튼 4회 클릭 (각 10개 * 5 페이지 = 50개 리뷰)

        more_cnt += 1 
        try:
            
            driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a')
            time.sleep(1)

            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(0.1)
            # driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
            
            driver.execute_script('return document.querySelector("#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a").click()')
            time.sleep(1) 
                        
        except NoSuchElementException:
            print("-모든 리뷰 더보기 완료-")
            break    


    #식당 평균 별점 수집
    rating = 0
    try:
        rating = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div.place_section_content > div > div.Xj_yJ > span.m7jAR.ohonc > em').text
        print('식당 평균 별점 : ', rating)
    except:
        pass
    
    
    #리뷰 데이터 스크래핑을 위한 html 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    
    try:
        #키워드 리뷰가 아닌 리뷰글 리스트 검색
        review_lists = soup.select('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.place_section_content > ul > li')
                
        print('총 리뷰 수 : ', len(review_lists))
        
        #리뷰 수가 0이 아닌 경우 리뷰 수집
        reviews = [] # 리뷰리스트 초기화
        print('리뷰를 수집합니다')
        if len(review_lists) > 0 :
            for j, review in enumerate(review_lists):
                
                print('.',end='')
                try:
                    #내용 더보기가 있는 경우 내용 더보기를 눌러주기
                    try:
                        review.find(' div.ZZ4OK > a > span.rvCSr > svg')
                        more_content = review.select(' div.ZZ4OK > a > span.rvCSr > svg')
                        more_content.click()
                        time.sleep(1)
   
                        #리뷰 정보
                        user_review = review.select(' div.ZZ4OK > a > span')  
                
                        #리뷰 정보가 있는 경우
                        if len(user_review) > 0:
                            rev = user_review[0].text
                            reviews.append(rev)  
                        time.sleep(1)
                
                    
                    
                    except:
                        #리뷰 정보
                        user_review = review.select(' div.ZZ4OK.IwhtZ > a > span')
                        
        
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        if len(user_review) > 0:
                            rev = user_review[0].text
                            reviews.append(rev)   
                        time.sleep(0.1) 
                        
                except NoSuchElementException:
                    print('리뷰 텍스트가 인식되지 않음')
                    continue
                
            
                
        else:
            print('리뷰 선택자가 인식되지 않음')
            time.sleep(1)
        

    # 리뷰가 없는 경우        
    except NoSuchElementException: 
        time.sleep(1)           
        print("리뷰가 존재하지 않음")
        

    #스크래핑한 데이터를 데이터 프레임으로 만들기
    name = df['name'][i]
    category = df['category'][i]
    address = df['address'][i]
    roadAddress = df['roadAddress'][i]
    rate = rating
    review = '<>'.join(reviews) # 리스트-> <> 구분자 합치기
    
    rev_list = [name, category, address, roadAddress, rate, review, home_url]

    col = ['name', 'category', 'address', 'roadAddress', 'rate', 'review', 'home_url']
    df2 = pd.DataFrame(rev_list).T
    df2.columns = col


    if os.path.exists(save_file):
        # 데이터프레임 저장 append mode
        df2.to_csv(save_file, encoding='utf-8-sig', mode='a', index=False, header=False)
    else:
        # 최초 1회만 실행  # 저장할 df 형태 생성/초기화 
        print('! 기존 csv 파일이 없어 새로 생성됨')
        new_df = pd.DataFrame(columns=col)
        new_df.to_csv(save_file, encoding='utf-8-sig', index=False)
    
    print('데이터 프레임 생성/저장 완료!')



driver.close()

***** 진행상황 : 1/5
현재 수집중인 식당 :  토담골
식당 평균 별점 :  4.39
총 리뷰 수 :  50
리뷰를 수집합니다
..................................................! 기존 csv 파일이 없어 새로 생성됨
데이터 프레임 생성/저장 완료!
***** 진행상황 : 2/5
현재 수집중인 식당 :  현정이네식당
-모든 리뷰 더보기 완료-
총 리뷰 수 :  0
리뷰를 수집합니다
리뷰 선택자가 인식되지 않음
데이터 프레임 생성/저장 완료!
***** 진행상황 : 3/5
현재 수집중인 식당 :  이목식당
식당 평균 별점 :  4.89
총 리뷰 수 :  50
리뷰를 수집합니다
..................................................데이터 프레임 생성/저장 완료!
***** 진행상황 : 4/5
현재 수집중인 식당 :  엄마밥상
식당 평균 별점 :  4.28
총 리뷰 수 :  50
리뷰를 수집합니다
..................................................데이터 프레임 생성/저장 완료!
***** 진행상황 : 5/5
현재 수집중인 식당 :  행복한밥상
-모든 리뷰 더보기 완료-
총 리뷰 수 :  16
리뷰를 수집합니다
................데이터 프레임 생성/저장 완료!


In [130]:
df2

,name,category,address,roadAddress,rate,review,home_url
0,행복한밥상,"한식>백반,가정식",경기도 수원시 장안구 조원동 774-2,경기도 수원시 장안구 송정로 164,0,"제육볶음 먹었는데, 메인인 제육부터 반찬이며 야채며 된장찌개며 너무 대접받는 느낌으...",https://m.place.naver.com/restaurant/114277209...


In [131]:
save_file

'gg_review_sample.csv'

In [132]:
pd.read_csv(save_file, encoding='utf-8-sig')

,name,category,address,roadAddress,rate,review,home_url
0,현정이네식당,"한식>백반,가정식",경기도 수원시 장안구 파장동 595-16,경기도 수원시 장안구 경수대로1031번길 8 한국일보,0.00,NaN,https://m.place.naver.com/restaurant/163255868...
1,이목식당,"한식>백반,가정식",경기도 수원시 장안구 이목동 210-10,경기도 수원시 장안구 장안로 433,4.89,사장님이 그날 그날 좋은메뉴도 추천해주시고 술 종류도 엄청 다양해\n서 고르는 재미...,https://m.place.naver.com/restaurant/112393957...
2,엄마밥상,"한식>백반,가정식",경기도 수원시 장안구 파장동 267-1,경기도 수원시 장안구 경수대로1078번길 17-1 경진만물전자,4.28,잘 먹었습니다 \n감사합니다<>매콤해물순두부 먹었습니다 국산 콩으로 만들어졌으며 매...,https://m.place.naver.com/restaurant/20118729/...
3,행복한밥상,"한식>백반,가정식",경기도 수원시 장안구 조원동 774-2,경기도 수원시 장안구 송정로 164,0.00,"제육볶음 먹었는데, 메인인 제육부터 반찬이며 야채며 된장찌개며 너무 대접받는 느낌으...",https://m.place.naver.com/restaurant/114277209...


In [133]:
# 나중에 여러번 append 로 저장하다보면 중복 제거 해줘야 함
# 메모리 오류날 수 있음

result_df = pd.read_csv(save_file,encoding='utf-8-sig')
print('중복제거 이전 길이:', len(result_df))

result_df_d = result_df.drop_duplicates(['name', 'category', 'address']) # 중복제거
result_df_d = result_df_d.reset_index(drop=True)
print('중복제거 이후 길이:', len(result_df_d))



중복제거 이전 길이: 4
중복제거 이후 길이: 4


In [134]:
result_df_d.to_csv(f'gg_review_sample_result.csv', encoding='utf-8-sig', index=False)


In [135]:
print('완료')

완료
